In [144]:
import sys
print(sys.executable)

/opt/miniconda/envs/ksa/bin/python


In [159]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import pandas as pd
from tqdm import tqdm

# Suppress specific warnings
warnings.filterwarnings("ignore", category=FutureWarning, module='pandas', lineno=11)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [160]:
data = 'not imputed'
idprov = '63'

In [161]:
## Get ID and Year 
years = range(2021, 2024)
numbers = range(1, 31)
year_id_ = []
for year in years:
    for number in numbers:
        year_id_.append(f"{year}_{str(number).zfill(2)}")
year_id_[:5]

vh_list = [f"VH_{i}" for i in range(30, -1, -1)]
print(vh_list)

# vv_list = [f"VV_{i}" for i in range(30, -1, -1)]
# print(vv_list)


['VH_30', 'VH_29', 'VH_28', 'VH_27', 'VH_26', 'VH_25', 'VH_24', 'VH_23', 'VH_22', 'VH_21', 'VH_20', 'VH_19', 'VH_18', 'VH_17', 'VH_16', 'VH_15', 'VH_14', 'VH_13', 'VH_12', 'VH_11', 'VH_10', 'VH_9', 'VH_8', 'VH_7', 'VH_6', 'VH_5', 'VH_4', 'VH_3', 'VH_2', 'VH_1', 'VH_0']


In [162]:
## Prep DF
df_all_wide = pd.DataFrame(columns=["idpoint"]+year_id_)
# df_all_wide

df_bridging_citra = pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object', sheet_name="periode_to_date")
# df_bridging_citra.head(2)

df_bridging_ksa = pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object')
# df_bridging_ksa.head(2)

In [163]:
def get_df_values(data, idprov, mgrs):
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
            df_values = pickle.load(f)    
        df_values["VH"] = df_values.Sigma0_VH_db
        # df_values["VV"] = df_values.Sigma0_VV_db
        # df_values.drop(["Sigma0_VH_db","Sigma0_VV_db"],axis=True, inplace=True)
        df_values.drop(["Sigma0_VH_db"],axis=True, inplace=True)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/'+mgrs+'_imputed_data.pkl', 'rb') as f:
            df_values = pickle.load(f) 
        # print(df_values.columns)
        df_values["VH"] = df_values.Sigma0_VH_db_imputation
        # df_values["VV"] = df_values.Sigma0_VV_db_imputation
        # df_values.drop(["Sigma0_VH_db_imputation","Sigma0_VV_db_imputation"],axis=True, inplace=True)
        df_values.drop(["Sigma0_VH_db_imputation"],axis=True, inplace=True)
    return df_values

def reformat_to_wide(df_values, band, df_bridging_citra, df_all_wide):
    
    df_values["periode_start"] = df_values.periode.str[4:8]
    df_values["periode_end"] = df_values.periode.str[-4:]
    df_values["is_kabisat"] = 0
    
    df_values = df_values.merge(df_bridging_citra, left_on=['periode_start','periode_end','is_kabisat'],  right_on=['periode_start','periode_end','is_kabisat'])
    df_values['year_id_per_image'] = df_values.periode.str[:4]+"_"+df_values.id_per_image.astype("str").str.zfill(2)
    
    df_VH_wide_res = df_values.sort_values('year_id_per_image').pivot(index='idpoint', columns='year_id_per_image', values=band).reset_index()
    df_VH_wide_res = pd.concat([df_all_wide, df_VH_wide_res], axis=0)
    df_VH_wide_res['idsubsegmen'] = df_VH_wide_res.idpoint.str[:-3]
    return df_VH_wide_res

In [164]:
# df_label = pd.read_csv("/data/raw/processed/relabelled_data_ksa.csv")

# df_label = df_label.merge(df_bridging_ksa.query("is_kabisat == 0"), how='left', left_on='bulan', right_on='obs_in_a_year')
# df_label['year_id_per_image'] = "20"+df_label.tahun.astype("str")+"_"+df_label.id_per_image.astype("str").str.zfill(2)
# df_label.head()

In [165]:
df_repair = pd.read_csv("/data/raw/processed/data_ksa_long_relabeled_repair.csv")
df_repair = df_repair[["tahun","bulan",'idsubsegmen_repair', 'idsubsegmen_old', 'nth', 'id_x', 'observation',
       'class']].merge(df_bridging_ksa.query("is_kabisat == 0"), how='left', left_on='bulan', right_on='obs_in_a_year')
df_repair['year_id_per_image'] = "20"+df_repair.tahun.astype("str")+"_"+df_repair.id_per_image.astype("str").str.zfill(2)
print(df_repair.shape)
df_repair.head(2)

(4322808, 14)


,tahun,bulan,idsubsegmen_repair,idsubsegmen_old,nth,id_x,observation,class,is_kabisat,obs_in_a_year,id_per_image,periode_start,periode_end,year_id_per_image
0,22,1,110101001A1,110101001A1,0,110101001A1,8.0,NV,0,1,3,0125,0205,2022_03
1,22,1,110101001A2,110101001B1,0,110101001A2,4.0,H,0,1,3,0125,0205,2022_03


In [166]:
year_id_per_image_ = df_label.year_id_per_image.unique()
year_id_per_image_


array(['2022_03', '2022_05', '2022_08', '2022_10', '2022_13', '2022_15',
       '2022_18', '2022_20', '2022_23', '2022_25', '2022_28', '2022_30',
       '2023_03', '2023_05', '2023_08', '2023_10', '2023_13', '2023_15',
       '2023_18', '2023_20', '2023_23', '2023_25', '2023_28', '2023_30'],
      dtype=object)

In [167]:
# df_full.loc[df_full.idsubsegmen_old == '351217018C2']

In [168]:
df_full = df_VH_wide[['idpoint','idsubsegmen']].copy()
df_full = df_full.merge(df_repair[['idsubsegmen_repair','idsubsegmen_old','tahun', 'bulan', 'observation', 'class', 'year_id_per_image']], how="left", left_on = "idsubsegmen", right_on="idsubsegmen_old")
df_full['kdpoint'] = df_full['idpoint'].str[-3:]
df_full['idpoint'] = df_full['idsubsegmen_repair'] +  df_full['kdpoint']
df_full['idsubsegmen'] = df_full['idsubsegmen_repair']
df_full.head()

,idpoint,idsubsegmen,idsubsegmen_repair,idsubsegmen_old,tahun,bulan,observation,class,year_id_per_image,kdpoint
0,320201009A1#01,320201009A1,320201009A1,320201009A1,22.0,1,8.0,NV,2022_03,#01
1,320201009A1#01,320201009A1,320201009A1,320201009A1,22.0,2,8.0,NV,2022_05,#01
2,320201009A1#01,320201009A1,320201009A1,320201009A1,22.0,3,8.0,NV,2022_08,#01
3,320201009A1#01,320201009A1,320201009A1,320201009A1,22.0,4,8.0,NV,2022_10,#01
4,320201009A1#01,320201009A1,320201009A1,320201009A1,22.0,5,8.0,NV,2022_13,#01


In [169]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x for x in mgrs_done_ if ".pkl" in x]
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]
mgrs_done_

['50MMA',
 '50MLB',
 '50MLC',
 '50MLA',
 '50MMB',
 '50MKA',
 '50MKC',
 '50MKB',
 '50MMC']

In [170]:
for mgrs in tqdm(mgrs_done_):
    df_values = get_df_values(data, idprov, mgrs)
    df_VH_wide = reformat_to_wide(df_values, "VH", df_bridging_citra, df_all_wide)

    # df_full = df_VH_wide[['idpoint','idsubsegmen']].copy()
    # df_full = df_full.merge(df_label[['id_x','tahun', 'bulan', 'obs', 'class', 'year_id_per_image']], how="left", left_on = "idsubsegmen", right_on="id_x")
    
    df_full = df_VH_wide[['idpoint','idsubsegmen']].copy()
    df_full = df_full.merge(df_repair[['idsubsegmen_repair','idsubsegmen_old','tahun', 'bulan', 'observation', 'class', 'year_id_per_image']], how="left", left_on = "idsubsegmen", right_on="idsubsegmen_old")
    df_full['kdpoint'] = df_full['idpoint'].str[-3:]
    df_full['idpoint'] = df_full['idsubsegmen_repair'] +  df_full['kdpoint']
    df_full['idsubsegmen'] = df_full['idsubsegmen_repair']

    df_wide_full = pd.DataFrame()
    for yi in year_id_per_image_:
        df_tmp = df_full.loc[df_full.year_id_per_image == yi]
        df_tmp.loc[:,'MGRS'] = mgrs
        ind = df_VH_wide.columns.to_list().index(yi)+1
        df_wide_tmp = pd.concat([df_VH_wide.iloc[:,0:1], df_VH_wide.iloc[:,ind-31:ind]], axis=1)
        df_wide_res = df_tmp.merge(df_wide_tmp, how='left', left_on='idpoint', right_on='idpoint')
        df_wide_res.columns.values[-31:] = vh_list
        df_wide_full = pd.concat([df_wide_full,df_wide_res], axis=0) 
        # break
    # break
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/wide_data_'+mgrs+'.pkl', 'wb') as f:
            pickle.dump(df_wide_full, f)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/wide_data_'+mgrs+'.pkl', 'wb') as f:
            pickle.dump(df_wide_full, f)
    df_wide_full
    # break

100%|██████████| 9/9 [03:40<00:00, 24.51s/it]


In [171]:
df_wide_full

,idpoint,idsubsegmen,idsubsegmen_repair,idsubsegmen_old,tahun,bulan,observation,class,year_id_per_image,kdpoint,...,VH_9,VH_8,VH_7,VH_6,VH_5,VH_4,VH_3,VH_2,VH_1,VH_0
0,630215002A1#01,630215002A1,630215002A1,630215002A1,22,1,7.0,NP,2022_03,#01,...,-13.458991,-14.534101,NaN,-11.453424,NaN,-12.742740,NaN,-13.295194,-14.961252,NaN
1,630215002A1#02,630215002A1,630215002A1,630215002A1,22,1,7.0,NP,2022_03,#02,...,-15.793771,-13.768745,NaN,-14.049233,NaN,-12.841273,NaN,-11.602156,-11.690808,NaN
2,630215002A1#03,630215002A1,630215002A1,630215002A1,22,1,7.0,NP,2022_03,#03,...,-13.865894,-14.213884,NaN,-13.924355,NaN,-13.085409,NaN,-10.336298,-12.325940,NaN
3,630215002A1#04,630215002A1,630215002A1,630215002A1,22,1,7.0,NP,2022_03,#04,...,-12.957108,-13.740613,NaN,-13.464873,NaN,-11.626237,NaN,-11.597968,-12.688139,NaN
4,630215002A1#05,630215002A1,630215002A1,630215002A1,22,1,7.0,NP,2022_03,#05,...,-13.086611,-11.971251,NaN,-12.862750,NaN,-12.216900,NaN,-12.512072,-12.915163,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,630220005C3#21,630220005C3,630220005C3,630220005C3,23,12,7.0,NP,2023_30,#21,...,NaN,-12.113348,NaN,-14.291942,-12.226669,NaN,-13.461864,NaN,-12.981815,NaN
896,630220005C3#22,630220005C3,630220005C3,630220005C3,23,12,7.0,NP,2023_30,#22,...,NaN,-12.421253,NaN,-15.396297,-13.152828,NaN,-16.117857,NaN,-13.648221,NaN
897,630220005C3#23,630220005C3,630220005C3,630220005C3,23,12,7.0,NP,2023_30,#23,...,NaN,-12.522540,NaN,-12.259372,-13.016039,NaN,-15.004348,NaN,-14.216055,NaN
898,630220005C3#24,630220005C3,630220005C3,630220005C3,23,12,7.0,NP,2023_30,#24,...,NaN,-12.791525,NaN,-14.161838,-13.963500,NaN,-15.232998,NaN,-14.208263,NaN


## samplign_VH / sampling_VV

In [172]:
# recode_dict = {
#     'V1': '1.0',
#     'V2': '2.0',
#     'G': '3.0',
#     'H': '4.0',
#     'PL': '5.0',
#     'P': '99.0',
#     'NP': '6.0',
#     'NV': '7.0',
#     'BL': '0.0'
# }

# mgrs_ = ['48MXT', '48MYT', '48MXU', '48MXS', '48MYS', '48MZT', '48MZS',
#        '49MAN', '49MAM', '49MBM', '49MBN', '49MAP', '49MBP', '48MZU',
#        '48MYU']
# mgrs

recode_dict = {
    'V1':1,
    'V2':2,
    'G':3,
    'H':4,
    'BP':0,
    'PL':5,
    'BPL':0,
    'NP':6,
    'NV':6,
    None:99
}

In [173]:
df_sampling['class'].unique()


array(['NV', 'V2', 'G', 'V1', 'BPL', 'NP', 'H', 'PL', 'BP'], dtype=object)

In [174]:
for mgrs in tqdm(mgrs_done_):
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/wide_data_'+mgrs+'.pkl', 'rb') as f:
            df_sampling = pickle.load(f)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/01_imputation/wide_data/wide_data_'+mgrs+'.pkl', 'rb') as f:
            df_sampling = pickle.load(f)

    df_sampling['observation'] = df_sampling['class'].replace(recode_dict)
    df_sampling = df_sampling.loc[df_sampling.observation != 'P']
    # df_sampling = df_sampling.loc[df_sampling.observation != '99.0']
    df_sampling = df_sampling.loc[df_sampling.observation != 99]
    df_sampling['idsegment'] = df_sampling['idsubsegmen'].str[:-2]
    df_sampling.rename(columns={'idsubsegmen': 'idsubsegment'}, inplace=True)
    df_sampling.rename(columns={'bulan': 'nth'}, inplace=True)
    df_sampling.rename(columns={'year_id_per_image': 'periode'}, inplace=True)
    
    df_sampling = df_sampling[['idpoint','idsubsegment','idsegment','nth','periode',
     'observation','class', 'MGRS', 'VH_30', 'VH_29', 'VH_28', 'VH_27', 'VH_26', 'VH_25',
     'VH_24', 'VH_23', 'VH_22', 'VH_21', 'VH_20', 'VH_19', 'VH_18', 'VH_17',
     'VH_16', 'VH_15', 'VH_14', 'VH_13', 'VH_12', 'VH_11', 'VH_10', 'VH_9',
     'VH_8', 'VH_7', 'VH_6', 'VH_5', 'VH_4', 'VH_3', 'VH_2', 'VH_1', 'VH_0']]

    # df_sampling = df_sampling[['idpoint','idsubsegment','idsegment','nth','periode',
    #  'observation', 'class','MGRS', 'VV_30', 'VV_29', 'VV_28', 'VV_27', 'VV_26', 'VV_25',
    #  'VV_24', 'VV_23', 'VV_22', 'VV_21', 'VV_20', 'VV_19', 'VV_18', 'VV_17',
    #  'VV_16', 'VV_15', 'VV_14', 'VV_13', 'VV_12', 'VV_11', 'VV_10', 'VV_9',
    #  'VV_8', 'VV_7', 'VV_6', 'VV_5', 'VV_4', 'VV_3', 'VV_2', 'VV_1', 'VV_0']]
    
    if(data == 'not imputed'):
        with open('/data/ksa/03_Sampling/data-wide/'+idprov+'/sampling_VH_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_sampling, f)
    elif(data == 'imputed'):
        with open('/data/ksa/04_Data_Preprocessing/'+idprov+'/wide_data/sampling_VH_'+mgrs+'.pkl', 'wb') as f:
            pickle.dump(df_sampling, f)

    # if(data == 'not imputed'):
    #     with open('/data/ksa/03_Sampling/data-wide/32/sampling_VV_'+mgrs+'.pkl', 'wb') as f:
    #             pickle.dump(df_sampling, f)
    # elif(data == 'imputed'):
    #     with open('/data/ksa/04_Data_Preprocessing/32/01_imputation/wide_data/sampling_VV_'+mgrs+'.pkl', 'wb') as f:
    #         pickle.dump(df_sampling, f)
            
    
    # break


100%|██████████| 9/9 [00:21<00:00,  2.43s/it]


In [175]:
df_sampling.observation.unique()

array([6, 4, 2, 1, 0, 3, 5], dtype=object)

In [176]:
df_sampling.groupby(['observation','class']).size()

observation  class
0            BP       4750
             BPL      1100
1            V1        450
2            V2        375
3            G         375
4            H         500
5            PL        600
6            NP       7700
             NV       3000
dtype: int64